# Translation (OpenAI api)
Steps: 
1. Data cleaning
    1. read data
    2. extract "input" column of text only
2. Translation
    1. request OpenAI api
    2. translate into another column
3. Evaluation
    1. compare translation and original text, evaluate translation quality

In [22]:
import numpy as np
import os, pandas as pd

# read data
df = pd.read_csv('picDescription.csv')
print(df.head())


   Unnamed: 0   ID  picIndex  eff_word input  pic_num     rt  rt_adjust  \
0         801  101        77         1    对称        1  77197      77197   
1         802  101        77         2  色彩柔和        1  27238      27238   
2         803  101        77         3  金碧辉煌        1  19496      19496   
3         804  101        77         4  暮色如雪        1  57735      57735   
4         805  101        77         5  湖光辉映        1  16576      16576   

   foreign  set      logrt    normrt   coordCP    pickCP  
0        0    1  11.254116  2.797662  0.330589  0.402072  
1        0    1  10.212368  1.359849  0.330589  0.402072  
2        0    1   9.877965  0.898307  0.330589  0.402072  
3        0    1  10.963619  2.396720  0.330589  0.402072  
4        0    1   9.715711  0.674366  0.330589  0.402072  


In [23]:
# Add a word_ID column based on row index
df['word_id'] = df.index + 1  # Start IDs from 1

In [24]:
# Extract "input" column into a new df
words_df = df[['word_id','input']]
print(words_df.head())

   word_id input
0        1    对称
1        2  色彩柔和
2        3  金碧辉煌
3        4  暮色如雪
4        5  湖光辉映


In [33]:
# Biuld a test set
test_df = words_df.sample(n=5, random_state=5).reset_index(drop=True)
print(test_df)

   word_id input
0    40593    钦佩
1    15884  高耸入云
2    51274    历史
3    49902    恬淡
4    92099    修补


In [ ]:
# install tenacity and openai
# !pip install tenacity openai

In [ ]:
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential
from openai import OpenAI

In [ ]:
# 0) setup
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"  # or use a .env file
client = OpenAI()

In [27]:
# 1) load data and set columns
# words_df
df = test_df.copy()
col_in = "input"
col_out = "output_EN"

In [ ]:
# 2) a style guardrails
STYLE = (
    "Translate to natural, formal English. "
    "Ensure the translation is accurate and preserves the original meaning. "
    # "Regardless of the number of input characters, translate each input into 1 English word."
)